In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
M, N, K = 1000, 100, 100
A = np.arange(M * K).astype(np.int32)
B = np.arange(K * N).astype(np.int32)
C1 = np.empty((M * N), dtype=np.int32)

In [2]:
from datetime import datetime
start = datetime.now()

for i in range(M):
    for j in range(N):
        C1[i*N + j] = 0
        for k in range(K):
            C1[i*N + j] += A[i*K + k] * B[k*N + j]
print(datetime.now() - start)

0:00:11.052835


In [3]:
# Изменение порядка циклов дает прирост в 3 секунды
start = datetime.now()
for i in range(M):
    for j in range(N):
        C1[i * N + j] = 0
    for k in range(K):
        k_N = k * N
        a = A[i * K + k]
        i_N = i * N
        for j in range(N):
            C1[i_N + j] += a * B[k_N + j]
            
print(datetime.now() - start)

0:00:07.789281


In [4]:
C2 = np.zeros((M, N))
A2 = A.reshape(M, K).astype(np.int32)
B2 = B.reshape(K, N).astype(np.int32)
C2 = (A2 @ B2).flatten()

from numpy import testing
# Выдаст ошибку, если матрицы не совпадают
testing.assert_array_equal(C2, C1)

In [5]:
# с использованием библиотеки numpy 

%%timeit
C2 = np.zeros((M, N))
A2 = A.reshape(M, K).astype(np.int32)
B2 = B.reshape(K, N).astype(np.int32)
C2 = (A2 @ B2).flatten()

13.4 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
# с использованием CUDA

from numba import cuda

@cuda.jit
def matmul(A, B, C):

    grid_column, grid_row = cuda.grid(2)
    stride_column, stride_row = cuda.gridsize(2)
    
    for data_row in range(grid_row, A.shape[0], stride_row):
        for data_column in range(grid_column, B.shape[1], stride_column):
            sum = 0
            for i in range(A.shape[1]):
                sum += A[data_row][i] * B[i][data_column]
                
            C[data_row][data_column] = sum


A3 = cuda.to_device(A.reshape(M, K))
B3 = cuda.to_device(B.reshape(K, N))
C3 = cuda.to_device(np.zeros((M, N)).astype(np.int32))

threads_per_block = (32,32)
blocks = (32,32) 

matmul[blocks,  threads_per_block](A3, B3, C3)

# Выдаст ошибку, если матрицы не совпадают
testing.assert_array_equal(C3.copy_to_host().flatten(), C2)


In [7]:
# прирост незначительный по сравнению с реализацией с помощью numpy, 
# потому что учитывается время перенеса матриц на граффический процессор
%%timeit
A3 = cuda.to_device(A.reshape(M, K))
B3 = cuda.to_device(B.reshape(K, N))
C3 = cuda.to_device(np.zeros((M, N)).astype(np.int32))

matmul[blocks,  threads_per_block](A3, B3, C3)
C3 = C3.copy_to_host()

2.26 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
C3 = cuda.to_device(np.zeros((M, N)).astype(np.int32))

In [9]:
# чистое время вычисления без переноса данных
%%timeit
matmul[blocks,  threads_per_block](A3, B3, C3)

194 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
